<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Pset 4: Problems Using Numerical Simulation</h1>


<a name='section_4_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P4.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_4_1">P4.1 Double Pendulum</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_4_1">P4.1 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_4_3">P4.3 Maxwell-Boltzmann Distribution from Molecular Simulation</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_4_3">P4.3 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_4_4">P4.4 Ideal Gas Law from Molecular Simulation</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_4_4">P4.4 Problems</a></td>
    </tr>
</table>



<h3>Learning Objectives</h3>

In these problems, we will explore several examples utilizing numerical simulation. For each case, there will be several questions which lead to building a full-fledged simulation of the physical system or computational task under consideration.

In the first part, we will simulate a dynamical system, the double-pendulum. This simple system exhibits rich behavior and it one of the prototypical examples of chaos.

In the second part, we will investigate the flow of momentum on a lattice using a Lattice-Boltzmann simulation.

The third and fourth parts, we will focus on a 2D ideal gas simulation through particle-level simulation. We will derive famous distributions and relations such as Boltzmann distribution, Gay-Lussac's law, Avogadro's Law, Boyle's law, and by combining them all, ideal gas relation in two dimensions.

Finally, we will simulate a galaxy and, from this simuilation, we will then compute rotation curves.


<h3>Importing Libraries</h3>

Before beginning, run the cell below to import the relevant libraries for this notebook.

In [ ]:
#>>>RUN: P4.0-runcell00

#install the following:

#!pip install imageio
#!pip3 install torch torchvision torchaudio
#!pip install lmfit

In [ ]:
#>>>RUN: P4.0-runcell01

import itertools
from IPython.display import HTML
from scipy.integrate import odeint
import matplotlib.patches as patches
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter
import imageio
from IPython.display import Image
import random

<h3>Setting Default Figure Parameters</h3>

The following code cell sets default values for figure parameters.

In [ ]:
#>>>RUN: P4.0-runcell02

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title


<a name='section_4_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P4.1 Double Pendulum</h2>    

| [Top](#section_4_0) | [Previous Section](#section_4_0) | [Problems](#problems_4_1) | [Next Section](#section_4_3) |


<h3>Overview</h3>

The double pendulum is a system in classical mechanics where one pendulum is attached to the end of another pendulum. Its equations of motion are usually expressed using the Lagrangian mechanics and solved numerically, which is what we will do in this problem

In [ ]:
#Below is a reworking of 
#https://scipython.com/blog/the-double-pendulum/
#https://github.com/zaman13/Double-Pendulum-Motion-Animation/blob/master/Python%20Code/Double_Pendulum_v1.py

## Setting up the differential equations to be solved

def theta1ddot(theta1,theta1dot,theta2,theta2dot,m,ell,g):
        st1t2=np.sin(theta1-theta2)
        ct1t2=np.cos(theta1-theta2)
        term1=-m*ell**2*st1t2*(ct1t2*theta1dot**2 + theta2dot**2)
        term2= m*g*ell*(-2*np.sin(theta1)+np.sin(theta2)*ct1t2)
        denom=m*ell**2*(1+st1t2**2)
        return (term1+term2)/denom

def theta2ddot(theta1,theta1dot,theta2,theta2dot,m,ell,g):
        st1t2=np.sin(theta1-theta2)
        ct1t2=np.cos(theta1-theta2)
        term1=m*ell**2*st1t2*(ct1t2*theta2dot**2 + 2*theta1dot**2)
        term2=2*m*g*ell*(-np.sin(theta2)+np.sin(theta1)*ct1t2)
        denom=m*ell**2*(1+st1t2**2)
        return (term1+term2)/denom


def ode(y, t, l, m, g):
    """Return the first derivatives of y = theta1, z1, theta2, z2."""
    theta1, theta1dot, theta2, theta2dot = y
    t1ddt=theta1ddot(theta1,theta1dot,theta2,theta2dot,m,l,g)
    t2ddt=theta2ddot(theta1,theta1dot,theta2,theta2dot,m,l,g)
    return theta1dot,t1ddt,theta2dot,t2ddt

def energy(y,l,m,g):
    theta1, theta1dot, theta2, theta2dot = y.T # Note the.T transposes it so we can unpack
    ct1t2=np.cos(theta1-theta2)
    U=-2*m*g*l*np.cos(theta1)-m*g*l*np.cos(theta2)
    K=m*l**2*theta1dot**2+m*l**2/2*(theta2dot**2+2*theta1dot*theta2dot*ct1t2)
    return K+U

## set up pendulum parameters
l=5#m
m=1#kg
g=9.8#kgm/s^2

<a name='problems_4_1'></a>     

| [Top](#section_4_0) | [Restart Section](#section_4_1) | [Next Section](#section_4_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.1.1</span>

Later in this problem when doing the numerics, one parameter we can easily calculate is the energy of the system. Before doing any numerics, is energy conserved in this system?


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.1.2</span>

Report the total energy for systems with the following initial conditions (consider four separate cases and report your answer as a list of number with precision 1e-1): `[energy case1, energy case2, energy case3, energy case4]`

Case 1: no initial amplitude, no initial angular velocity\
Case 2: small initial amplitude of pendulum 1 only (other initial conditions zero): `theta1=0.02*pi`\
Case 3: larger initial amplitude of both pendula (angular velocities zero): `theta1=0.3*pi`, `theta2=0.2*pi`\
Case 4: nearly vertical (angular velocities zero): `theta1=0.9*pi`, `theta2=*pi`

In [ ]:
#>>>PROBLEM: P4.1.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#define arrays for the 4 cases by setting the initial conditions in y0,
#e.g., y0_case1=np.array([theta1_case1, theta1dot_case1, theta2_case1, theta2dot_case1])

y0_case1 = #YOUR CODE HERE
y0_case2 = #YOUR CODE HERE
y0_case3 = #YOUR CODE HERE
y0_case4 = #YOUR CODE HERE

#print the energy for each case, using `energy(y,l,m,g)`

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.1.3</span>

Before doing a simulation, it can be useful to think about test cases where you know the answer to check your simulation outputs. One simple way to decompose a system is into normal modes. 


Which of the following statements is true about the normal modes of a double-pendulum system?

A) There is only one normal mode, which is called the out-of-phase mode.\
B) There are two normal modes, which are called the in-phase mode and the out-of-phase mode.\
C) There are three normal modes, which are called the in-phase mode, the out-of-phase mode, and the chaotic mode.\
D) There are four normal modes, which are called the in-phase mode, the out-of-phase mode, the chaotic mode, and the stable mode.



<h3>Run the Simulation</h3>

Now run the cells below to see the simulation.

In [ ]:
## Solve the differenial equation with odeint and make a video of the bob's motion
def run(y0):

    # y0 is the initial conditions theta1, theta1dot, theta2, theta2dot = y
    # for example y0=np.array([np.pi*0.1,0,np.pi*0.2,0])
    
    #Time
    tmax=30#s
    dt=1e-2#s
    t=np.arange(0, tmax+dt, dt)

    #The works
    y = odeint(ode, y0, t, args=(l,m,g))
    #print(y)
    
    #the check
    en=energy(y,l,m,g)
    
    return t,y,en
    
    
#===========Plotting 
def make_plot(i,y,l,g,ax,fig,images):
    plt.cla()
    # Plot and save an image of the double pendulum configuration for time
    # point i.
    # The pendulum rods.
    theta1=y[:,0]
    theta2=y[:,2]
    x1 = l  * np.sin(theta1)
    y1 = -l * np.cos(theta1)
    x2 = x1 + l * np.sin(theta2)
    y2 = y1 - l * np.cos(theta2)
    ax.plot([0, x1[i], x2[i]], [0, y1[i], y2[i]], lw=2, c='k')

    # Circles representing the anchor point of rod 1, and bobs 1 and 2.
    r = 0.35
    c0 = patches.Circle((0, 0), r/2, fc='k', zorder=10)
    c1 = patches.Circle((x1[i], y1[i]), r, fc='b', ec='b', zorder=10)
    c2 = patches.Circle((x2[i], y2[i]), r, fc='r', ec='r', zorder=10)
    ax.add_patch(c0)
    ax.add_patch(c1)
    ax.add_patch(c2)

    # The trail will be divided into ns segments and plotted as a fading line.
    ns = 20
    trail_secs = 1
    dt=1e-2#s
    max_trail = int(1 / dt)
    s = max_trail // ns

    for j in range(ns):
        imin = i - (ns-j)*s
        if imin < 0:
            continue
        imax = imin + s + 1
        # The fading looks better if we square the fractional length along the
        # trail.
        alpha = (j/ns)**2
        ax.plot(x2[imin:imax], y2[imin:imax], c='r', solid_capstyle='butt',
                lw=2, alpha=alpha)

    # Centre the image on the fixed anchor point, and ensure the axes are equal
    ax.set_xlim(-l-l-r, l+l+r)
    ax.set_ylim(-l-l-r, l+l+r)
    ax.set_aspect('equal', adjustable='box')
    plt.axis('off')
    fps = 10
    di = int(1/fps/dt)
    # plt.savefig('frames/_img{:04d}.png'.format(i//di), dpi=72)
    # plt.cla()
    #plt.show()
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(image)

In [ ]:
#set the initial conditions and run the simulation

#Define a new y0, or test cases defined previously
#e.g.,y0=y0_case1
#y0 = #YOUR CODE HERE

y0=y0_case3

t,y,en=run(y0)

fps = 1000
dt=.0001
di = int(1/fps/dt)
fig = plt.figure(figsize=(8.3333, 6.25), dpi=72)
ax = fig.add_subplot(111)

# Create the output of the simulation, stored in the variable `images`
images=[]
for i in range(0, t.size, di):
    #print(i // di, '/', t.size // di)
    make_plot(i,y,l,g,ax,fig,images)
    

# Comment out plt.close() to view the last image from the simulation, if desired
plt.close()

#save the gif
imageio.mimsave('./double_p.gif', images, fps=10)

In [ ]:
# VIEW THE GIF
# comment out the line and run again to hide

#Image(open('double_p.gif','rb').read())

In [ ]:
## plot the bobs' angles vs time and energy
## use previously defined y0, above

plt.plot(t,y[:,0],color='blue')
plt.plot(t,y[:,2],color='red')
plt.xlabel("Time [seconds]")
plt.ylabel(r"$\theta$")
plt.show()


plt.plot(t,en,color='blue')
plt.xlabel("Time [seconds]")
plt.ylabel("Energy")
plt.show()

fig = plt.figure(figsize=(8.3333, 6.25), dpi=72)
ax = fig.add_subplot(111)
make_plot(10,y,l,g,ax,fig,images)
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.1.4</span>

For each of the following initial conditions, describe the bob's motion. To do this, edit and run the relevant code above and analyze plots of $\theta$ vs. $t$ for each case.

<pre>
#initial conditions:
y0_case1=np.array([0,0,0,0])
y0_case2=np.array([np.pi*0.02,0,np.pi*0.0,0])
y0_case3=np.array([np.pi*0.3,0,np.pi*0.2,0])
y0_case4=np.array([np.pi*.9,0,np.pi*1,0]) 
</pre>

Select your answer for each case from the following options:

A) Stationary\
B) Each bob oscillates at a single frequency\
C) Each bob oscillates periodically and displays beats\
D) One bob oscillates periodically and one bob oscillates chaotically\
E) Both bobs oscillate chaotically


Enter your answer as a list of **uppercase** letters, corresponding to the correct choices for each set of initial conditions, i.e.: `[answer case1, answer case2, answer case3, answer case4 ]` should be entered as, e.g., `['A','B','C','D']`, where the letters are **capitalized** and listed as strings.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.1.5</span>


Is energy conserved during the evolution for each of the above initial conditions? Try plotting the energy in each case to determine your answer.

A) Yes.\
B) No, the energy fluctuates significantly as the pendula move.\
C) No, the energy decays significantly when the bobs are moving.

<!--start-block-->
<a name='section_4_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P4.3 Maxwell-Boltzmann Distribution from Molecular Simulation</h2>

| [Top](#section_4_0) | [Previous Section](#section_4_1) | [Problems](#problems_4_3) | [Next Section](#section_4_4) |

<h3>Overview</h3>

In this section, you will code the microscopic simulation of an ideal gas, where the particles move freely inside a stationary container without interacting with one another, except for very brief collisions in which they exchange energy and momentum with each other or with their thermal environment.

Then we will derive the <a href="https://en.wikipedia.org/wiki/Maxwell%E2%80%93Boltzmann_distribution" target="_blank">Maxwell-Boltzmann distribution</a>, which describes velocity distribution of these idela gas particles, using the microscopic simulation.




<a name='problems_4_3'></a>     

| [Top](#section_4_0) | [Restart Section](#section_4_3) | [Next Section](#section_4_4) |

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.3.1</span>

We know that for elastic collsion of a mass with a wall (stationary infinite mass object), the velocity component perpendicular to the wall flips direction, while the horizontal component stays the same.

<p align="center">
<img alt="Wall Collision" src="https://github.com/SangeonPark/psets/blob/main/pset4/images/wall_collision.png?raw=1" width="300"/>
</p>

$$v_{f, \perp} = -v_{i,\perp} $$

$$v_{f, \parallel} = v_{i,\parallel} $$


Complete the code for `wall_collision`, which takes as input `pos` and `vel` that represents position and velocity vectors of particles, and returns the velocity vectors after the collsion with four sides of the wall. Assume that both `pos` and `vel` have shape `[n_particles, 2]`.

Assume that the 2D box is a square with side length `L` equal to 1.

In [ ]:
#>>>PROBLEM: P4.3.1

def wall_collision(pos, vel):

    #collsion with the right wall
    vel[pos[:, 0] > 1, 0] = #Your code goes here

    #collision with the left wall
    vel[pos[:, 0] < 0, 0] = #Your code goes here

    #collision with the top wall
    vel[pos[:, 1] > 1, 1] = #Your code goes here

    #collsion with the bottom wall
    vel[pos[:, 1] < 0, 1] = #Your code goes here
    return vel


#-----------------------------------------------------
# CHECK YOUR RESULT

# Sample positions (x, y) of particles
pos_initial = np.array([
    [0.5, 0.5],  # Inside the box
    [1.5, 0.5],  # Colliding with the right wall
    [-0.5, 0.5], # Colliding with the left wall
    [0.5, 1.5],  # Colliding with the top wall
    [0.5, -0.5]  # Colliding with the bottom wall
])

# Sample velocities (vx, vy) of particles
vel_initial = np.array([
    [0.1, 0.1],  # Random velocity
    [0.2, 0.2],  # Random velocity
    [-0.3, -0.3],# Random velocity
    [0.4, 0.4],  # Random velocity
    [-0.5, -0.5] # Random velocity
])

print("Original Velocities:\n", vel_initial)

# Apply the wall_collision function to the sample data
updated_vel = wall_collision(pos_initial, vel_initial)

# Print the updated velocities after collision handling
print("\nUpdated Velocities after Collision Handling:\n", updated_vel)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.3.2</span>

Let's run a simulation of a single particle bouncing off four walls of a 2D box, using the above code you completed. The simulation will run in time steps `dt`, which is set to be a very small value.

First, in each time step, we handle the wall collision of the particle, and flip the component of the velocity vector using function `wall_collision` we coded above.

Then, we use the position update formula

$$\vec{x}_{i+1} = \vec{x}_{i} + dt \cdot \vec{v}_{i+1}$$

which is discretizing the continuous relation

$$\vec{v} = \frac{d\vec{x}}{dt} $$

We save all the simulated position and velocity vectors in arrays of shape `[time_steps, n_particles ,2]`, where `n_particles` is equal to 1 in this case since we are simulating a single particle, and the last dimension saves the `x` and the `y` component of that particle.

Check your result by printing the position and velocity of the particle in step `1111`.

In [ ]:
#>>>PROBLEM: P4.3.2

def simulate_wall(pos, vel, time_steps, dt):
    n_particles = pos.shape[0]
    pos_simulated = np.zeros((time_steps,n_particles, 2))
    vel_simulated = np.zeros((time_steps,n_particles, 2))

    pos_simulated[0] = pos
    vel_simulated[0] = vel

    #update the position and velocity
    for i in range(1,time_steps):
        #Your code here

    return pos_simulated, vel_simulated


#-----------------------------------------------------
# CHECK YOUR RESULT

pos_sim, vel_sim = simulate_wall(np.array([[0.5,0.5]]), np.array([[200.0,100.0]]), 8000, 0.0001)
print(pos_sim[1111][0], vel_sim[1111][0])

<h3>Run the simulation!</h3>

Run the code below to simulate a single particle bouncing off the walls.

In [ ]:
#run the line below, if you have not previously
pos_sim, vel_sim = simulate_wall(np.array([[0.5,0.5]]), np.array([[200.0,100.0]]), 8000, 0.0001)

fig, ax = plt.subplots(1,1,figsize=(5,5))
ax.clear()
vmin = 0
vmax = 1
ax.set_xlim(0,1)
ax.set_ylim(0,1)
markersize = 2 * 0.005 * ax.get_window_extent().width  / (vmax-vmin) * 72./fig.dpi
red, = ax.plot([], [], 'o', color='red', markersize=markersize)

def animate(i):
    xred, yred = pos_sim[i][:,0], pos_sim[i][:,1]
    red.set_data(xred, yred)
    return [red]


ani = animation.FuncAnimation(fig, animate, frames=500, interval=50, blit=True)
HTML(ani.to_jshtml())

<h3> Interparticle Collisions </h3>

Now let's write code that handles interparticle collisions. The ideal gas molecules don't interact with one another except for very brief collisions, in which they exchange energy and momentum with each other or with their thermal environment. For an ideal gas, all interparticle collisions will be elastic collsions.



### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.3.3</span>

For practical implementation, in each simulation step, we need to track which pair of particles collide. We will perform an elastic collision between two particles when they get closer than a distance of one diamter. Thus, in order to detect a collision, first we have to calculate the center of mass distances between each particles.

For bookkeeping indices, we give `n_particles` indices `0,1,...,n_particles-1`.

Given position vectors `position` of shape `[n_particles, 2]` and an array of indices corresponding to the particle pairs `idx_pair` of shape `[n_pair, 2]`, where each row is `[index_particle1, index_particle2]`, write a function that computes an array of shape `[n_pair]`, which is a pairwise distance of particles between every index pair of particles.  

For a given random `position` and `idx_pair` arrays below, report the resulting `distance_pairs` for the `0`, `23`, `234`, and `444`th entries.

In [ ]:
#>>>PROBLEM: P4.3.3

# helper function that calculates distance between particles of given index pair
def compute_distance_pairs(position, idx_pair):
    delta_x = #Your code goes here
    delta_y = #Your code goes here
    distances = #np.sqrt(delta_x**2 + delta_y**2)
    return distances


#-----------------------------------------------------
# CHECK YOUR RESULT

np.random.seed(0x98a09fe)
n_particles = 50 #number of particles
pos_initial = np.random.rand(n_particles, 2)
indices = np.arange(0,n_particles,1)
idx_pair = np.array(list(itertools.combinations(indices,2)))
print(compute_distance_pairs(pos_initial, idx_pair)[[0,23,234,444]])

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.3.4</span>

Now, given a distance pair array for a given array of index pairs, we need to calculate which pairs actually collide, meaning they satisfy the condition that the center of mass distance between a pair of particles is smaller than `2*particle_radius`.

Write a simple function that takes `distance_pairs` as an input, and returns the array of indices of `idx_pairs` corresponding to pairs that collide. Hint: this should really just use a `np.where()` function!

To check your answer, use the same random array of particles that you previously defined, the code for which is again included below. **Enter indices of pairs that collide, as a list**

In [ ]:
#>>>PROBLEM: P4.3.4

def which_pair_collides(distance_pairs):
    #Your code goes here
    pass


#-----------------------------------------------------
# CHECK YOUR RESULT

np.random.seed(0x98a09fe)
n_particles = 50 #number of particles
pos_initial = np.random.rand(n_particles, 2)
indices = np.arange(0,n_particles,1)
idx_pair = np.array(list(itertools.combinations(indices,2)))

particle_radius = 0.01
distance_pairs = compute_distance_pairs(pos_initial, idx_pair)
which_pair_collides(distance_pairs)

<h3>Elastic Collsion of Particles</h3>


<p align="center">
<img alt="unit circle wave decomposition" src="https://upload.wikimedia.org/wikipedia/commons/2/2c/Elastischer_sto%C3%9F_2D.gif" width="600"/>
</p>

>source: https://commons.wikimedia.org/wiki/File:Animation_zum_Verst%C3%A4ndnis_der_Fourierentwicklung.gif<br>
>attribution: PZim, CC BY-SA 3.0 <https://creativecommons.org/licenses/by-sa/3.0>, via Wikimedia Commons

    
When two particles get closer than 2 times the radius of the particle, we should perform elastic collision of particles.

In an elastic collision (conservation of energy + momentum + angular momentum, although in this case we neglect angular momentum), one can show

$$\vec{v}_1^{\text{new}} = \vec{v}_1 - \frac{(\vec{v}_1 - \vec{v}_2) \cdot (\vec{r}_1 - \vec{r}_2)}{|\vec{r}_1 - \vec{r}_2|^2} (\vec{r}_1 - \vec{r}_2)$$
$$\vec{v}_2^{\text{new}} = \vec{v}_2 - \frac{(\vec{v}_2 - \vec{v}_1) \cdot (\vec{r}_2 - \vec{r}_1)}{|\vec{r}_1 - \vec{r}_2|^2} (\vec{r}_2 - \vec{r}_1)$$

You can read more about it in <a href="https://en.wikipedia.org/wiki/Elastic_collision" target="_blank">Wikipedia</a> or any intro to mechanics class textbook.



### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.3.5</span>

After a collision event, what is the final velocity of each of the two spheres?

Complete the code for `pair_collsion` below, which takes `pos_1i`, `pos_2i`, `vel_1i` and `vel_2i` as inputs representing the position and velocity vectors of particles, and returns the velocity vectors after the collsion with four sides of the wall.

In [ ]:
#>>>PROBLEM: P4.3.5

def pair_collision(pos_1i, pos_2i, vel_1i, vel_2i):
    vel_1f = 0 #YOUR CODE HERE
    vel_2f = 0 #YOUR CODE HERE
    return vel_1f, vel_2f


#-----------------------------------------------------
# CHECK YOUR RESULT

# Sample initial positions (x, y) and velocities (vx, vy) of two particles
pos_1i = np.array([[0.5, 0.5],
                   [0.8, 0.8]])
pos_2i = np.array([[0.7, 0.6],
                   [0.4, 0.4]])

vel_1i = np.array([[0.1, 0.1],
                   [-0.1, -0.1]])
vel_2i = np.array([[-0.1, 0.1],
                   [0.1, -0.1]])


# Apply the pair_collision function to the sample data
vel_1f, vel_2f = pair_collision(pos_1i, pos_2i, vel_1i, vel_2i)

# Print the updated velocities after collision handling
print("Initial Velocities:")
print("Particle 1:", vel_1i)
print("Particle 2:", vel_2i)

print("\nUpdated Velocities after Collision Handling:")
print("Particle 1:", vel_1f)
print("Particle 2:", vel_2f)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.3.6</span>

Let's edit the `simulate` function we coded earlier to incorporate all particle-wise collisions.

The `idx_pair` variable keeps index pairs of every possible pair of particles.

We will handle pairwise collision as the first action in each loop, and wall collision afterwards.

Complete the function!

In [ ]:
#>>>PROBLEM: P4.3.6

def simulate(pos, vel, time_steps, dt):

    nparticles = pos.shape[0]
    indices = np.arange(0,n_particles,1)
    idx_pair = np.array(list(itertools.combinations(indices,2)))

    pos_simulated = np.zeros((time_steps, n_particles, 2))
    vel_simulated = np.zeros((time_steps, n_particles, 2))

    pos_simulated[0] = pos
    vel_simulated[0] = vel

    #update the position and velocity based on particle collisions
    #and wall collisions
    for i in range(1,time_steps):
        #Your code here

    return pos_simulated, vel_simulated

In [ ]:
#>>>SOLUTION: P4.3.6

def simulate(pos, vel, time_steps, dt):

    nparticles = pos.shape[0]
    indices = np.arange(0,n_particles,1)
    idx_pair = np.array(list(itertools.combinations(indices,2)))

    pos_simulated = np.zeros((time_steps, n_particles, 2))
    vel_simulated = np.zeros((time_steps, n_particles, 2))

    pos_simulated[0] = pos
    vel_simulated[0] = vel

    #update the position and velocity based on particle collisions
    #and wall collisions
    for i in range(1,time_steps):

        collision_idx = which_pair_collides(compute_distance_pairs(pos, idx_pair))
        v1f, v2f = pair_collision(pos[idx_pair[collision_idx][:,0]], pos[idx_pair[collision_idx][:,1]],
                                  vel[idx_pair[collision_idx][:,0]], vel[idx_pair[collision_idx][:,1]])

        vel[idx_pair[collision_idx][:,0]] = v1f
        vel[idx_pair[collision_idx][:,1]] = v2f

        vel = wall_collision(pos, vel)

        pos = pos + vel * dt

        vel_simulated[i] = vel
        pos_simulated[i] = pos

    return pos_simulated, vel_simulated

<h3>Setting Parameters of our Simulation</h3>

Now let's run this thing! It might take a few moments to complete...

In [ ]:
np.random.seed(4150032420)
n_particles = 65 #number of particles
particle_radius = 0.003 #radius of individual particles
L = 1 #side length of the box

pos_initial = np.random.rand(n_particles, 2) #inditial position of particles
# Setting up the initial 2D velocities of particles
vel_initial = np.zeros((n_particles,2))
vel_initial[pos_initial[:,0]>=L/2, 0] = -100
vel_initial[pos_initial[:,0]<L/2, 0] = 100
vel_initial[:, 1] = 0

pos_sim, vel_sim = simulate(pos_initial, vel_initial, 5000, 0.0001)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,5))
ax.clear()
vmin = 0
vmax = 1
ax.set_xlim(0,1)
ax.set_ylim(0,1)
markersize = 2 * particle_radius * ax.get_window_extent().width  / (vmax-vmin) * 72./fig.dpi
red, = ax.plot([], [], 'o', color='red', markersize=markersize)

def animate(i):
    xred, yred = pos_sim[i][:,0], pos_sim[i][:,1]
    red.set_data(xred, yred)
    return [red]



ani = animation.FuncAnimation(fig, animate, frames=500, interval=50, blit=True)
HTML(ani.to_jshtml())

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.3.7</span>

Now, let's run the simulation until it reaches thermal equilibrium, and then compare the final velocity distribution of the particles to the Maxwell-Boltzmann distribution in 2 Dimensions:

Using <a href="https://en.wikipedia.org/wiki/Equipartition_theorem" target="_blank">Equipartition Theorem</a> in thermal equilibrium, each degree of freedom contributes $\frac{1}{2}k_B T$ to the average energy.

In 2-dimensional space, there are 2 degrees of freedom:

* $d \times \frac{1}{2}k_B T = 2 \times \frac{1}{2}k_B T = KE_{avg} = \frac{1}{2}m\bar{v^2} \implies \boxed{\frac{m}{kT} = \frac{2}{\bar{v^2}}}$

* $\boxed{f(v) = \frac{m}{kT} v \exp\left(-\frac{m}{kT}\frac{v^2}{2} \right)}$


Let's set the initial velocity to $100 m/s$, and use this fact to to calculate the value of $\frac{m}{kT}$. Complete the function `maxwell_boltzmann(v)` with this information.

In [ ]:
#>>>PROBLEM: P4.3.7

def maxwell_boltzmann(v):
    fv = #YOUR CODE HERE
    return fv


#-----------------------------------------------------
# CHECK YOUR RESULT

v_test = np.linspace(0, 2000, 1001)
print(maxwell_boltzmann(v_test)[np.where(v_test==100)[0]])

<h3>Comparing Simulation to Theory</h3>

We will now run a simulation and compare to the predicted distribution. Let's increase the number of particles in order to populate a histogram more completely. This might take some time...

In [ ]:
#let's increase the number of particles significantly

np.random.seed(673720454)
n_particles = 400
particle_radius = 0.0002
v = np.linspace(0, 2000, 1001)

pos_initial = np.random.rand(n_particles, 2)
vel_initial = np.zeros((n_particles,2))
thetas = np.random.rand(n_particles)* 2 * np.pi
vel_initial[:, 0] = np.cos(thetas) * 100
vel_initial[:, 1] = np.sin(thetas) * 100

# This can take a while to run...
pos_sim, vel_sim = simulate(pos_initial, vel_initial, 200000, 0.0000008)

In [ ]:
bins = np.linspace(0,300,21)
plt.clf()
plt.figure()
plt.hist(np.sqrt(np.sum(vel_sim[-1]**2, axis=1)), bins=bins, density=True)
plt.plot(v,maxwell_boltzmann(v))
plt.xlabel('Velocity [m/s]')
plt.ylabel('# Particles')
plt.xlim(0,300)
#plt.show()

With some simple simulation, we were able to obtain Maxwell-Boltzmann distribution! Now let's turn that into an animation and see how the distribution evolves from the initial uniform velocity.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(9,4))
axes[0].clear()
vmin = 0
vmax = 1
axes[0].set_xlim(0,1)
axes[0].set_ylim(0,1)
axes[0].get_xaxis().set_visible(False)
axes[0].get_yaxis().set_visible(False)
markersize = 10*2 * particle_radius * axes[0].get_window_extent().width  / (vmax-vmin) * 72./fig.dpi
red, = axes[0].plot([], [], 'o', color='red', markersize=markersize)
n, bins, patches = axes[1].hist(np.sqrt(np.sum(vel_sim[0]**2, axis=1)), bins=bins, density=True)
axes[1].plot(v,maxwell_boltzmann(v))
axes[1].set_xlim(0,300)
axes[1].set_ylim(0,0.01)
axes[1].get_yaxis().set_visible(False)

def animate(i):
    xred, yred = pos_sim[i][:,0], pos_sim[i][:,1]
    red.set_data(xred, yred)
    hist, _ = np.histogram(np.sqrt(np.sum(vel_sim[i]**2, axis=1)), bins=bins, density=True)
    for i, patch in enumerate(patches):
        patch.set_height(hist[i])
    return [red]

from itertools import chain
ani_hist = animation.FuncAnimation(fig, animate, frames=range(0,200000,500), interval=50, blit=True)
#list(chain(range(100),range(100, 39500, 100), range(39500, 40000)))

HTML(ani_hist.to_jshtml())

<h3>Going Beyond: Other Energy Scales</h3>

Does this simulation hold for all energy scales (i.e., initial velocity distributions)? Is energy always conserved (try making plots of total energy vs. time)? Consider if there any changes that you need to make when simulating a system with starting velocity of 500m/s or 1000m/s.

<!--start-block-->
<a name='section_4_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P4.4 Ideal Gas Law from Molecular Simulation</h2>

| [Top](#section_4_0) | [Previous Section](#section_4_3) | [Problems](#problems_4_4) |

<h3>Overview</h3>

Now with the setup we just completed, we will derive ideal gas relations.

The ideal gas law combines Gay-Lussac's law, Avogadro's law, and Boyle's law. We will confirm each of them through our molecular simulation and, in order to do that, we will compute macroscopic quantities such as pressure based on the microscopic components.

In real life, it's easier to keep the pressure constant to atmospheric presure, and vary the volume. However, in simulations, it's easier to keep the volume constant and measure the pressure change.

Therefore, for the case of Avogadro's law, we will show a version where we keep the volume constant and vary the number of particles, while observing the pressure.  

<h3>Gay-Lussac's Law</h3>

Firstly, we will check Gay-Lussac's law, which states that the pressure exerted by a given mass and constant volume of an ideal gas on the sides of its container is directly proportional to its absolute temperature. In practice, we will keep the volume of the box and the number of particles in the simulation constant, and we will show that $P \propto T$. We then fit the data witha linear model to obtain the slope, $\frac{P}{T} = A$ where $A$ is some constant.


The pressure $P$ can be calculated as the following:

$$P = \frac{F}{4L}$$

where $F$ is the force and $L$ is the side length of the box. We just need to compute the imparted force. Noting that $F = \Delta p / \Delta t$, we can compute the force as follows:

 * We track particles that hit the walls of the box and compute the change of momentum $\Delta p_i = m \Delta v_i$ given to the box at each time step.
 * The mass of the individual particles is kept constant.
 * The force is then $F = \sum_i |\Delta p_i| / \Delta t$ where $\Delta t$ is the length of time we sum the momentum change, after reaching thermal equilibrium

Therefore,

 $$P = \frac{m \sum |\Delta v_i|}{4L \Delta t}$$

The temperate and the average temperature has relation by equipartition theorem as described above.

$$d \times \frac{1}{2}k_B T = 2 \times \frac{1}{2}k_B T = k_B T = KE_{avg} = \frac{1}{2}m\bar{v^2} $$

Therefore we get

 $$T = \frac{m \bar{v^2}}{2 k_B}$$

where $\bar{v^2}$ is the initial velocity magnitude that we set as a constant for every particle.

Now, we see that both $P$ and $T$ have dependence on $m$, the particle mass. Since they cancel out for $P/T$, we will in practice calculate $P/m$ and $T/m$

<a name='problems_4_4'></a>     

| [Top](#section_4_0) | [Restart Section](#section_4_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.4.1</span>

Write a function named `compute_delta_v` that takes 2 inputs: a velocity before collsion with the wall, `vel_i`, the velocity after collision, `vel_f`, and then returns the sum of the change of magnitude of all the collsions with the wall:

$$\sum_i |\Delta v_i|$$



In [ ]:
#>>>PROBLEM: P4.4.1

def compute_delta_v(vel_i, vel_f):
    result = #YOUR CODE HERE
    return result

<h3>Augmenting the Simulation</h3>

Let's add this to our simulation code to also keep track of $\Delta v$

In [ ]:
def simulate(pos, vel, time_steps, dt):


    nparticles = pos.shape[0]
    indices = np.arange(0,n_particles,1)
    idx_pair = np.array(list(itertools.combinations(indices,2)))

    pos_simulated = np.zeros((time_steps, n_particles, 2))
    vel_simulated = np.zeros((time_steps, n_particles, 2))
    dv = np.zeros((time_steps))

    pos_simulated[0] = pos
    vel_simulated[0] = vel
    dv[0] = 0

    for i in range(1,time_steps):

        collision_idx = which_pair_collides(compute_distance_pairs(pos, idx_pair))
        v1f, v2f = pair_collision(pos[idx_pair[collision_idx][:,0]], pos[idx_pair[collision_idx][:,1]],
                                  vel[idx_pair[collision_idx][:,0]], vel[idx_pair[collision_idx][:,1]])

        vel[idx_pair[collision_idx][:,0]] = v1f
        vel[idx_pair[collision_idx][:,1]] = v2f

        vel_before = vel.copy()
        vel = wall_collision(pos, vel)
        delta_v = compute_delta_v(vel_before, vel)
        dv[i] = delta_v

        pos = pos + vel * dt


        vel_simulated[i] = vel
        pos_simulated[i] = pos

    return pos_simulated, vel_simulated, dv



### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.4.2</span>

Now write a function named `compute_pressure` that takes 3 inputs: an array `dv_sim` which keeps track of the $\Delta v$ in each simulation step, `npoints` the number of time steps we want to intergrate from the simulation end, and `dt` the length of the time step, then returns the pressure using the formula given in the overview section:

$$P/m = \frac{ \sum |\Delta v_i|}{4L \Delta t}$$


In [ ]:
#>>>PROBLEM: P4.4.2

def compute_pressure(dv_sim, npoints, dt):
    result = #YOUR CODE HERE
    return result


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.4.3</span>

As a check on your answer, use the function `compute_pressure` in the loop provided and enter the pressure for the case when `v_magnitude` is equal to $300 m/s$. Use precision `1e-3`. This will take a while!

In [ ]:
#>>>PROBLEM: P4.4.3

np.random.seed(673720454)
n_particles = 400 #number of particles
pos_initial = np.random.rand(n_particles, 2)
particle_radius = 0.0002

npoints = 2000
simulation_steps = 40000
dt = 0.0000008

pressure_list = []
v_list = np.array([0, 300, 350, 400, 450])
for v_magnitude in v_list:
    np.random.seed(673720454)
    vel_initial = np.zeros((n_particles,2))
    thetas = np.random.rand(n_particles)* 2 * np.pi
    vel_initial[:, 0] = np.cos(thetas) * v_magnitude
    vel_initial[:, 1] = np.sin(thetas) * v_magnitude
    pos_sim, vel_sim, dv_sim = simulate(pos_initial, vel_initial, simulation_steps, dt)
    pressure = compute_pressure(dv_sim, npoints, dt)
    pressure_list.append(pressure)

print(pressure_list)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.4.4</span>

Write a function named `compute_temperature` that takes 1 input: an array `v_list` of magnitudes of velocities, and returns an array of temperatures using the formula given in the overview section. Use `k_B=1.38e-23` J/K.

 $$T/m = \frac{\bar{v^2}}{2 k_B}$$

Now we are able to make a plot of P vs. T, try it out!

In [ ]:
#>>>PROBLEM: P4.4.4

def compute_temperature(vel):
    result = #YOUR CODE HERE
    return result

#MAKE A PLOT
T = compute_temperature(v_list)
plt.plot(T, pressure_list)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.4.5</span>

Now let's use lmfit to get the slope of $P$ vs $T$ graph. Firstly, scale the $x$ axis by $10^{-27}$ and the $y$ axis by $10^{-7}$. Fit the data with a linear model.

Enter the slope of the fit after fitting. Report your answer with precision `1e-2`.

In [ ]:
#>>>PROBLEM: P4.4.5

import lmfit
from lmfit import Model

#YOUR CODE HERE

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.4.6</span>

As a next step, let's confirm the pressure version of Avogadro's law. In other words, let's show that with temperature and volume fixed, $P \propto N$, where $N$ is the number of particles.

Enter the value of the pressure when the number of particles is equal to 450. Use precision `1e-3`. This will take a while!

In [ ]:
#>>>PROBLEM: P4.4.6

np.random.seed(673720454)
n_particles = 400 #number of particles
pos_initial = np.random.rand(n_particles, 2)
particle_radius = 0.0002

npoints = 2000
simulation_steps = 40000
dt = 0.0000008
nparticles_list = [300,350,400,450,500]
pressure_list = []
v_magnitude= 500

for n_particles in nparticles_list:
    np.random.seed(673720454)
    #YOUR CODE HERE
    pressure_list.append(pressure)

print(pressure_list[3])

#make a plot!
plt.plot(nparticles_list, pressure_list)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.4.7</span>

Finally, let's check Boyle's Law. We have to show that when we fix the number of particles and temperature, $P \propto \frac{1}{V}$. In order to do that, we should be able to vary the volume of the simulation by changing the side length variable `L` in the code.

Edit the code for the function `wall_collision` so that you can vary the wall side length `L`. As before, assume that both `pos` and `vel` have shape `[n_particles, 2]`, but now allow the side length `L` to be a global variable, defined outside of this function.

Then, enter the pressure value when the area of the container is equal to $4$. Finally draw the plot of the plot $P$ vs $\frac{1}{V}$

In [ ]:
#>>>PROBLEM: P4.4.7

def wall_collision(pos, vel):
    #collsion with the right wall
    vel[pos[:, 0] > L, 0] = # Your code goes here.

    #collision with the left wall
    vel[pos[:, 0] < 0, 0] = # Your code goes here.

    #collision with the top wall
    vel[pos[:, 1] > L, 1] = # Your code goes here.

    #collsion with the bottom wall
    vel[pos[:, 1] < 0, 1] = # Your code goes here.
    return vel

#-----------------------------------------------------
# CHECK YOUR RESULT

L = 2.

# Sample positions (x, y) of particles
pos_initial = np.array([
    [0.5, 0.5],  # Inside the box
    [L+0.5, 0.5],  # Colliding with the right wall
    [L-0.5, 0.5], # Colliding with the left wall
    [0.5, L+0.5],  # Colliding with the top wall
    [0.5, L-0.5]  # Colliding with the bottom wall
])

# Sample velocities (vx, vy) of particles
vel_initial = np.array([
    [0.1, 0.1],  # Random velocity
    [0.2, 0.2],  # Random velocity
    [-0.3, -0.3],# Random velocity
    [0.4, 0.4],  # Random velocity
    [-0.5, -0.5] # Random velocity
])

print("Original Velocities:\n", vel_initial)

# Apply the wall_collision function to the sample data
updated_vel = wall_collision(pos_initial, vel_initial)

# Print the updated velocities after collision handling
print("\nUpdated Velocities after Collision Handling:\n", updated_vel)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.4.8</span>

Run the simulation below and enter the pressure value when the container has side length $L=2$. Draw the plot of the plot $P$ vs $\frac{1}{V}$. This may take a while!

In [ ]:
#>>>PROBLEM: P4.4.8

np.random.seed(673720454)
n_particles = 400 #number of particles
pos_initial = np.random.rand(n_particles, 2)
particle_radius = 0.0002

npoints = 2000
simulation_steps = 40000
dt = 0.0000008
pressure_list = []
L_list = np.array([1,1.5,2,2.5,3])

for L in L_list:
    vel_initial = np.zeros((n_particles,2))
    np.random.seed(673720454)
    thetas = np.random.rand(n_particles)* 2 * np.pi
    vel_initial[:, 0] = np.cos(thetas) * v_magnitude
    vel_initial[:, 1] = np.sin(thetas) * v_magnitude
    pos_sim, vel_sim, dv_sim = simulate(pos_initial, vel_initial, simulation_steps, dt)
    pressure = compute_pressure(dv_sim, npoints, dt)
    pressure_list.append(pressure)
    #print(pressure)
    
print(pressure_list[2])

#Make a plot!
plt.plot(1/(L_list**2), pressure_list)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 4.4.9</span>

We've confirmed that:
* with $N$ and $V$ fixed, $P \propto T$
* with $V$ and $T$ fixed, $P \propto N$
* with $T$ and $N$ fixed, $P \propto \frac{1}{V}$

Thus, combining them all, we get that $\frac{PV}{NT} = C$ where $C$ is a constant.
Rearranging this equation, $\frac{P}{T} = C\frac{N}{V}$

In Problem 4.4.5, by fitting we got the value of $\frac{P}{T}$ with 400 particles and box of side length $1$. Using this fit value, compute the value of $C$. Report your answer as a number in units of J/K `1e-23`, with precision `1e-3`.

Hint. With 400 particles and the volume(area) = $1\times 1 = 1$, the slope in problem 4.4.5 is $400C$

Is $C$ close to the value $k_B$ as we expect? Rearranging the equation, we get $PV = Nk_B T$, the ideal gas relation in two dimensions.
